# ML.Net - DataFrame-Getting Started

In [ ]:
#r "nuget:Microsoft.Data.Analysis,0.2.0"

In [ ]:
using Microsoft.Data.Analysis; 

In [8]:
#!time
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<int> ints = new PrimitiveDataFrameColumn<int>("Ints", 3); // Makes a column of length 3. Filled with nulls initially
StringDataFrameColumn strings = new StringDataFrameColumn("Strings", 3); // Makes a column of length 3. Filled with nulls initially

Wall time: 55,6532ms

In [9]:
// Append 3 values to dateTimes
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/02"));

In [10]:
DataFrame df = new DataFrame(dateTimes, ints, strings ); // This will throw if the columns are of different lengths

In [11]:
df

Columns,Rows
"[ [ 2019-01-01 00:00:00Z, 2019-01-01 00:00:00Z, 2019-01-02 00:00:00Z ], [ <null>, <null>, <null> ], [ <null>, <null>, <null> ] ]","[ [ 2019-01-01 00:00:00Z, <null>, <null> ], [ 2019-01-01 00:00:00Z, <null>, <null> ], [ 2019-01-02 00:00:00Z, <null>, <null> ] ]"


In [12]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [13]:
df

0,2019-01-01 00:00:00Z,<null>,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-02 00:00:00Z,<null>,<null>


In [14]:
// To change a value directly through df
df[0, 1] = 10; // 0 is the rowIndex, and 1 is the columnIndex. This sets the 0th value in the Ints columns to 10
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-02 00:00:00Z,<null>,<null>


In [15]:
// Modify ints and strings columns by indexing
ints[1] = 100;
strings[1] = "Foo!";
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,100,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


In [16]:
// Indexing can throw when types don't match.
// ints[1] = "this will throw because I am a string";  
// Info can be used to figure out the type of data in a column. 
df.Info()

0,DataType,System.DateTime,System.Int32,System.String
1,Length (excluding null values),3,2,3


In [17]:
// Add 5 to ints through the DataFrame
df.Columns["Ints"].Add(5, inPlace: true);
df

0,2019-01-01 00:00:00Z,15,<null>
1,2019-01-01 00:00:00Z,105,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


In [18]:
// We can also use binary operators. Binary operators produce a copy, so assign it back to our Ints column 
df.Columns["Ints"] = (ints / 5) * 100;
df

0,2019-01-01 00:00:00Z,300,<null>
1,2019-01-01 00:00:00Z,2100,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


In [19]:
// Fill nulls in our columns, if any. Ints[2], Strings[0] and Strings[1] are null
df.Columns["Ints"].FillNulls(-1, inPlace: true);
df.Columns["Strings"].FillNulls("Bar", inPlace: true);
df

0,2019-01-01 00:00:00Z,300,Bar
1,2019-01-01 00:00:00Z,2100,Foo!
2,2019-01-02 00:00:00Z,-1,Bar


In [20]:
// To inspect the first row
DataFrameRow row0 = df.Rows[0];
row0

index Date Day DayOfWeek DayOfYear Hour Kind Millisecond Minute Month Second Ticks TimeOfDay Year 0 2019-01-01 00:00:00Z 1 Tuesday 1 0 Unspecified 0 0 1 0 636818976000000000 00:00:00 2019 1 300 2 Bar

In [21]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrameRow>.Register((dataFrameRow, writer) =>
{
    var cells = new List<IHtmlContent>();
    cells.Add(td(i));
    foreach (var obj in dataFrameRow)
    {
        cells.Add(td(obj));
    }
    
    var t = table(
        tbody(
            cells));
    
    writer.Write(t);
}, "text/html");

In [22]:
row0

2019-01-01 00:00:00Z 300 Bar

In [23]:
// Filter rows based on equality
PrimitiveDataFrameColumn<bool> boolFilter = df.Columns["Strings"].ElementwiseEquals("Bar");
boolFilter

index,value
0,True
1,False
2,True


In [24]:
DataFrame filtered = df.Filter(boolFilter);
filtered

0,2019-01-01 00:00:00Z,300,Bar
1,2019-01-02 00:00:00Z,-1,Bar


In [31]:
// Sort our dataframe using the Ints column
var sorted = df.Columns["Ints"];
df.Sort(sorted);
sorted

Unhandled exception: (3,4): error CS1061: ‘DataFrame’ não contém uma definição para "Sort" e não foi possível encontrar nenhum método de extensão "Sort" que aceite um primeiro argumento do tipo ‘DataFrame’ (você está se esquecendo de usar uma diretiva ou uma referência de assembly?)

In [21]:
// GroupBy 
GroupBy groupBy = df.GroupBy("DateTimes");
// Count of values in each group
DataFrame groupCounts = groupBy.Count();
groupCounts

0,2019-01-01 00:00:00Z,2,2
1,2019-01-02 00:00:00Z,1,1


In [22]:
// Alternatively find the sum of the values in each group in Ints
DataFrame intsGroupSum = groupBy.Sum("Ints");
intsGroupSum

0,2019-01-01 00:00:00Z,2400
1,2019-01-02 00:00:00Z,-1


In [23]:
using XPlot.Plotly;
using System.Linq;

In [24]:
#r "nuget:MathNet.Numerics,4.9.0"

Installed package MathNet.Numerics version 4.9.0

In [25]:
using MathNet.Numerics.Distributions;
double mean = 0;
double stdDev = 0.1;

MathNet.Numerics.Distributions.Normal normalDist = new Normal(mean, stdDev);

In [26]:
PrimitiveDataFrameColumn<double> doubles = new PrimitiveDataFrameColumn<double>("Normal Distribution", normalDist.Samples().Take(1000));
display(Chart.Plot(
    new Graph.Histogram()
    {
        x = doubles,
        nbinsx = 30
    }
));